# Feature  Engineering

This is the Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [87]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
PATH_DATA = '../../data/laptop_price/'

## Get The Data

In [3]:
df_raw = pd.read_csv(PATH_DATA + 'laptop_price_interim.csv')

In [4]:
df_raw.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Inches2,Weight2,Ram2,Product2,Company2
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69,13.3,1.37,8.0,MacBook,Apple
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94,13.3,1.34,8.0,Macbook,Apple
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00,15.6,1.86,8.0,250,HP
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45,15.4,1.83,16.0,MacBook,Apple
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60,13.3,1.37,8.0,MacBook,Apple


In [5]:
df_raw.shape

(1303, 18)

In [6]:
### predictor
col_num = ['Inches2','Weight2','Ram2'] ## -->pure numbers
col_cat = ['Company2','Product2','TypeName','OpSys'] ## --> categorical
col_campuran = ['ScreenResolution','Cpu','Memory','Gpu'] ## --> bisa diparsing
other_cols = list(set(df_raw.columns)-set(col_num+col_cat+col_campuran)) ## --> others

In [7]:
other_cols

['Product', 'Ram', 'Weight', 'laptop_ID', 'Inches', 'Price_euros', 'Company']

In [8]:
## Predictor --> column2 / feature2 yang digunakan untuk memprediksi

In [9]:
col_target = ['Price_euros']

## Train Test

In [14]:
df_raw.shape[0]*0.8

1042.4

In [25]:
id_train = df_raw.sample(1042, random_state=145)['laptop_ID'].values

In [27]:
df_raw['is_train']=df_raw.laptop_ID.isin(id_train).astype(int)

## Feature Campuran


In [40]:
df_raw[col_campuran].sample(8)

,ScreenResolution,Cpu,Memory,Gpu
1249,Quad HD+ / Touchscreen 3200x1800,Intel Core i5 7Y54 1.2GHz,256GB SSD,Intel HD Graphics 615
795,IPS Panel Touchscreen 2560x1440,Intel Core i5 7200U 2.5GHz,256GB SSD,Intel HD Graphics 620
446,1366x768,Intel Core i5 7200U 2.5GHz,500GB HDD,Intel HD Graphics 620
623,1366x768,Intel Core i5 7200U 2.5GHz,256GB SSD,Intel HD Graphics 620
1283,1366x768,Intel Core i7 6500U 2.5GHz,500GB HDD,Nvidia GeForce 920M
386,IPS Panel Full HD 1920x1080,Intel Core i3 7100U 2.4GHz,128GB SSD,Intel HD Graphics 620
1274,1366x768,Intel Celeron Dual Core N3050 1.6GHz,500GB HDD,Intel HD Graphics
667,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,1TB HDD,AMD Radeon 530


In [41]:
## Memory --> SSD or Not 
## ScreenResolution --> Full HD, IPS Panel, Common
## cpu --> berapa GHZ

In [52]:
def screen_func(x):
    x1 = x.lower()
    if 'full hd' in x1:
        return 1
    elif 'ips panel' in x1:
        return 2
    else:
        return 0

In [68]:
df_raw['is_ssd'] = df_raw.Memory.apply(lambda x : 1 if 'SSD' in x else 0).astype(int)
df_raw['screen_type'] = df_raw.ScreenResolution.apply(screen_func).astype(float)
df_raw['cpu_size'] = df_raw.Cpu.apply(lambda x: x[-6:-3].replace(' ','').replace('U2','2').replace('P3','3')).astype(float)

In [69]:
col_num = ['Inches2','Weight2','Ram2','cpu_size','is_ssd','screen_type'] ## -->pure numbers
col_cat = ['Company2','Product2','TypeName','OpSys'] ## --> categorical

In [71]:
# cat = 'is_ssd'

# plt.figure(figsize = (15,5))
# sns.boxplot(data = df_raw, y = col_target[0], x = cat)
# plt.show()

## Col Numeric Transformation

In [171]:
XX = df_raw[df_raw.is_train == 1][col_num]

SS_scaler = StandardScaler()
SS_scaler.fit(XX)

StandardScaler()

In [172]:
col_num2 = [c+'_ss' for c in col_num]
df_raw2_num = pd.DataFrame(SS_scaler.transform(df_raw[col_num]), columns = col_num2)
df_raw2_num['laptop_ID'] = df_raw['laptop_ID']
df_raw2_num['is_train'] = df_raw['is_train']

In [173]:
col_num2

['Inches2_ss',
 'Weight2_ss',
 'Ram2_ss',
 'cpu_size_ss',
 'is_ssd_ss',
 'screen_type_ss']

## Col Categoric

In [184]:
col_cat

['Company2', 'Product2', 'TypeName', 'OpSys']

In [185]:
df_raw[col_cat].describe()

,Company2,Product2,TypeName,OpSys
count,1303,1303,1303,1303
unique,9,226,6,9
top,Dell,Inspiron,Notebook,Windows 10
freq,297,141,727,1072


In [186]:
col_cat_temp = ['Company2','TypeName','OpSys']
cat = col_cat_temp[1]

In [187]:
col_cat2 = []
df_raw2_cat = []
for cat in col_cat_temp:
    OH = OneHotEncoder()
    XX = df_raw[cat].values.reshape(-1,1)
    OH.fit(XX)

    col_temp = [cat+'_'+c for c in OH.categories_]
    df_temp = pd.DataFrame(OH.transform(XX).toarray(), columns = list(col_temp[0]))
    col_cat2 += list(col_temp[0])
    df_raw2_cat.append(df_temp)

In [188]:
df_raw2_cat = pd.concat(df_raw2_cat,axis = 1)
df_raw2_cat['laptop_ID'] = df_raw['laptop_ID']
# df_raw2_cat['is_train'] = df_raw['is_train']

## Summary Feat Eng

In [189]:
col_num2

['Inches2_ss',
 'Weight2_ss',
 'Ram2_ss',
 'cpu_size_ss',
 'is_ssd_ss',
 'screen_type_ss']

In [190]:
col_cat2

['Company2_Acer',
 'Company2_Apple',
 'Company2_Asus',
 'Company2_Dell',
 'Company2_HP',
 'Company2_Lenovo',
 'Company2_MSI',
 'Company2_Toshiba',
 'Company2_others',
 'TypeName_2 in 1 Convertible',
 'TypeName_Gaming',
 'TypeName_Netbook',
 'TypeName_Notebook',
 'TypeName_Ultrabook',
 'TypeName_Workstation',
 'OpSys_Android',
 'OpSys_Chrome OS',
 'OpSys_Linux',
 'OpSys_Mac OS X',
 'OpSys_No OS',
 'OpSys_Windows 10',
 'OpSys_Windows 10 S',
 'OpSys_Windows 7',
 'OpSys_macOS']

In [191]:
col_target

['Price_euros']

In [192]:
df_raw2_cat.head()

,Company2_Acer,Company2_Apple,Company2_Asus,Company2_Dell,Company2_HP,Company2_Lenovo,Company2_MSI,Company2_Toshiba,Company2_others,TypeName_2 in 1 Convertible,...,OpSys_Android,OpSys_Chrome OS,OpSys_Linux,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS,laptop_ID
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5


In [193]:
df_raw2_num.head()

,Inches2_ss,Weight2_ss,Ram2_ss,cpu_size_ss,is_ssd_ss,screen_type_ss,laptop_ID,is_train
0,-1.231169,-1.014448,-0.087182,0.042451,0.745133,2.287662,1,1
1,-1.231169,-1.059224,-0.087182,-0.881391,-1.342041,-1.396641,2,1
2,0.398827,-0.283106,-0.087182,0.411987,0.745133,0.445511,3,1
3,0.257089,-0.327882,1.520678,0.781524,0.745133,2.287662,4,1
4,-1.231169,-1.014448,-0.087182,1.520597,0.745133,2.287662,5,1


In [199]:
df_interim = pd.merge(df_raw2_num, df_raw2_cat, on = 'laptop_ID', how = 'left')
df_interim = pd.merge(df_interim, df_raw[['laptop_ID']+col_target], on = 'laptop_ID', how = 'left')

In [200]:
df_interim.columns

Index(['Inches2_ss', 'Weight2_ss', 'Ram2_ss', 'cpu_size_ss', 'is_ssd_ss',
       'screen_type_ss', 'laptop_ID', 'is_train', 'Company2_Acer',
       'Company2_Apple', 'Company2_Asus', 'Company2_Dell', 'Company2_HP',
       'Company2_Lenovo', 'Company2_MSI', 'Company2_Toshiba',
       'Company2_others', 'TypeName_2 in 1 Convertible', 'TypeName_Gaming',
       'TypeName_Netbook', 'TypeName_Notebook', 'TypeName_Ultrabook',
       'TypeName_Workstation', 'OpSys_Android', 'OpSys_Chrome OS',
       'OpSys_Linux', 'OpSys_Mac OS X', 'OpSys_No OS', 'OpSys_Windows 10',
       'OpSys_Windows 10 S', 'OpSys_Windows 7', 'OpSys_macOS', 'Price_euros'],
      dtype='object')

In [202]:
df_interim.to_csv(PATH_DATA+'data_interim.csv',index = False)